In [1]:
from models.composer.generate import RNNGenerator
import numpy as np
import tensorflow as tf
from tensorflow.python.ops import array_ops
from data.preprocess import load_POP909_dataset_completor

configs = {
    'model_path': '/home/richhiey/Desktop/workspace/projects/virtual_musicians/cache/test_model5',
    'vocab_size': 389,
    'emb_size': 128,
    'lstm_units': 512,
    'dense_units': 64,
    'max_timesteps': 256,
    'rnn': {'num_units': 512, 'num_layers': 1, 'unit_type': 'lstm'},
}

g = RNNGenerator(configs)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(16, 256)]               0         
_________________________________________________________________
embedding (Embedding)        (16, 256, 128)            49792     
_________________________________________________________________
rnn (RNN)                    (16, 256, 512)            1312768   
_________________________________________________________________
dense (Dense)                (16, 256, 389)            199557    
Total params: 1,562,117
Trainable params: 1,562,117
Non-trainable params: 0
_________________________________________________________________
Initializing from scratch.


In [2]:
DATA_PATH = '/home/richhiey/Desktop/workspace/projects/virtual_musicians/data-store/POP909-Dataset/data_process/pop909-event-token.npy'
data = np.load(DATA_PATH, allow_pickle=True)
print(len(data))
melodies = [d['MELODY'] for d in data]
dataset = load_POP909_dataset_completor(DATA_PATH, 'MELODY', 2)


print(dataset)
for x in dataset:
    print(x)
    break
    

#g.train(dataset, {'num_epochs':1, 'print_every': 3, 'save_every': 1000,})

909
<ShuffleDataset shapes: ((None, 1, None), (None, 1, None)), types: (tf.int64, tf.int64)>


InvalidArgumentError: TypeError: `generator` yielded an element that did not match the expected structure. The expected structure was (tf.int64, tf.int64), but the yielded element was tf.Tensor([[355 355 355 ...  73 291 201]], shape=(1, 2221), dtype=int32).
Traceback (most recent call last):

  File "/home/richhiey/.local/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 833, in generator_py_func
    flattened_values = nest.flatten_up_to(output_types, values)

  File "/home/richhiey/.local/lib/python3.8/site-packages/tensorflow/python/data/util/nest.py", line 396, in flatten_up_to
    assert_shallow_structure(shallow_tree, input_tree)

  File "/home/richhiey/.local/lib/python3.8/site-packages/tensorflow/python/data/util/nest.py", line 297, in assert_shallow_structure
    raise TypeError(

TypeError: If shallow structure is a sequence, input must also be a sequence. Input has type: <class 'tensorflow.python.framework.ops.EagerTensor'>.


During handling of the